In [ ]:
import models
import numpy as np
import pandas as pd
from numpy import sqrt, exp, pi, power, tanh, vectorize
from scipy.integrate import odeint
import matplotlib.pyplot as plt

In [ ]:
folder = ''

In [ ]:
input_file = pd.read_csv(folder+'input_irradiance_mel.csv', sep=";", decimal=",")
irradiance_mel = input_file[['hours', 'irradiance_mel']]
illuminance_mel = input_file[['hours', 'illuminance_mel']]

In [ ]:
output_file = pd.read_csv(folder+"output.csv", sep=";", decimal=",")
n = output_file.shape[0]

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def irradiance(t):
    t = t/3600
    t = t % 24
    #print("Time:",t) # for debugging
    if ((t < 8.0) or (t > 20.0)): 
        E_emel = 0.01
    elif ((t < 9.0) or (t > 16.0)): 
        E_emel = max(irradiance_mel.irradiance_mel)
    else:
        idx = find_nearest(irradiance_mel.hours, t)
        E_emel = irradiance_mel.irradiance_mel[idx]
    #print("Irradiance:",E_emel) # for debugging
    return E_emel

irradiance_v = vectorize(irradiance)

In [ ]:
# initial conditions 
y0 = [1.5, -15.0, 13.0, 0.04, -1.28, 0.0, -0.055]
# y0 = [-4.497078351339751, -0.07000000000052416, 13.235369640119554, -0.07243371889456507, -0.7498943126792074, 0.998695292851908, 0.0]
# y0 = [-3.9849559306245554, -0.06999999999999991, 13.443772572885331, -0.8152711832774777, 0.10984121127149143, 0.08493027667594716, -0.03543102672658369]
# y0 = [2.902150366506143, -14.159798106868699, 13.299816271267845, -0.7966368638117368, 0.2637919920821084, 0.06249713611782928, -0.05188881098166024]
# y0 = [ 1.888050284627664, -10.000676037696763, 12.547896533553217, -1.0889180236787013, -0.2654388779780049, 0.06231185031284664, -0.05499507929393252]
# y0 = [ 1.745102761767178, -9.50905050894146, 12.40606970294775, -1.076264422103, -0.09355980932875388, 0.056699021756750685, -0.054995079291803216]

In [ ]:
t = np.linspace(0,72*60*60,n)
(sol,temp) = odeint(models.model, y0, t, args = (irradiance, '2020',), full_output = True)

V_v = sol[:, 0]
V_m = sol[:, 1]
H = sol[:, 2]
X = sol[:, 3]
Y = sol[:, 4]
P = sol[:, 5]
Theta_L = sol[:, 6]

In [ ]:
# define initial condition
idx = find_nearest(H, 13.3)
print("[ {}, {}, {}, {}, {}, {}, {}]".format(V_v[idx],V_m[idx],H[idx],X[idx],Y[idx],P[idx],Theta_L[idx]))


In [ ]:
# define initial condition
idx = find_nearest(t/3600, 48)
print("[ {}, {}, {}, {}, {}, {}, {}]".format(V_v[idx],V_m[idx],H[idx],X[idx],Y[idx],P[idx],Theta_L[idx]))

In [ ]:
t_hours = t/3600
C = models.circadian_drive_v(X,Y)
AM = models.alertness_measure_v(C, H, Theta_L)

In [ ]:
plt.plot(t_hours, C, 'darkorange', label='C(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(output_file.time, output_file.KSS, 'b', label='KSS(t)')
plt.plot(t_hours, AM, 'darkorange', label='AM_KSS(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(output_file.time, output_file.I_mel, 'b', label='I_mel(t), target')
plt.plot(t_hours, irradiance_v(t)/0.0013262, 'darkorange', label='I_mel(t), input conversion')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('KSS', color=color)
ax1.plot(t_hours, AM, color=color)
ax1.plot(output_file.time, output_file.KSS, color=color, linestyle='dashed')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel('Irradiance', color=color)  # we already handled the x-label with ax1
ax2.plot(t_hours, irradiance_v(t), color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
plt.plot(t_hours, V_v, 'b', label='V_v(t)')
plt.plot(t_hours, V_m, 'g', label='V_m(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(t_hours, H, 'r', label='H(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(t_hours, X, 'c', label='X(t)')
plt.plot(t_hours, Y, 'm', label='Y(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(t_hours, P, 'y', label='P(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
plt.plot(t_hours, Theta_L, 'b', label='Theta_L(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()